In [1]:
import numpy as np
import numba
import quantecon as qe
import matplotlib.pyplot as plt
import datetime
import interpolation

In [2]:
beta=0.96
alpha=0.36
delta=0.08
a_max = 25.
b=3.

In [3]:
#rho=0.2
rho=0.4

#mu=0
mu=3.
#mu=5

#sigma=0
sigma=0.4
#sigma=0.6
#sigma=0.9

In [4]:
x = qe.markov.approximation.tauchen(rho,sigma*(1-rho**2)**0.5, m = 3,n = 7)
l_grid = np.exp(x.state_values)
l_dis = x.stationary_distributions
L = np.sum(l_grid * l_dis)
l_min=l_grid[0]
piel = x.P

In [5]:
def optim_sv(phi, a_max, a, l, tras, full, r, w, mu, beta):
    err = 1.
    n = 0
    
    a_low = -phi
    a_up = a_max
    
    while err>5e-4 and n <=25:
        ap=0.5*(a_low+a_up)
        err=((1+r)*a+w*l-ap) ** (-mu) - beta * (1+r)* (((1+r)*ap + w*l_grid - full(ap)) ** (-mu)) @ tras
        if err>0:
            a_up = ap
        else:
            a_low = ap
        
        n+=1
    return ap

In [19]:
def plc_iter(phi, a_max, beta, mu, piel, r, w, l_grid):
    
    a_grid = np.linspace(-phi, a_max, 601, dtype=np.float64)
    drn = np.ones((601,7), dtype=np.float64)*12.
    
    def obj_fct(ap, a, l, tras):
        res = ((1+r)*a+w*l-ap) ** (-mu) - beta * (1+r)* (((1+r)*ap + w*l_grid - full(ap)) ** (-mu)) @tras
        return res
    n=0
    err=1.    
    
    while err > 1e-3 and n <= 1000:
        dr0=drn.copy()
        full = si.interp1d(a_grid, drn, axis=0, fill_value='extrapolate')
        for i in range(601):
            for j in range(7):
                if obj_fct(-phi, a_grid[i], l_grid[j], piel[j,:])>=0:
                    drn[i,j] = -phi
                elif obj_fct(a_max, a_grid[i], l_grid[j], piel[j,:])<=0:
                    drn[i,j] = a_max
                else:
                    drn[i,j] = optim_sv(phi, a_max, a_grid[i], l_grid[j], piel[j,:], full, r, w, mu, beta)
    
        if n % 10 == 0:
            err = np.max(np.abs(drn-dr0))

        n+=1
    
    if n==1000:
        print('plc_over')
    
    return drn

In [20]:
k = (((0.36+delta)/(alpha * L**(1-alpha)))) ** (1/(alpha-1))
w = (1-alpha) * (k**alpha) * (L**(-alpha))

plc=plc_iter(-0.5, 25., 0.96, 3., piel, 0.36, w, l_grid)

array([[0.5  , 0.5  , 0.5  , ..., 0.5  , 0.5  , 0.5  ],
       [0.5  , 0.5  , 0.5  , ..., 0.5  , 0.5  , 0.5  ],
       [0.5  , 0.5  , 0.5  , ..., 0.5  , 0.5  , 0.5  ],
       ...,
       [6.625, 6.625, 6.625, ..., 6.625, 6.625, 6.625],
       [6.625, 6.625, 6.625, ..., 6.625, 6.625, 6.625],
       [6.625, 6.625, 6.625, ..., 6.625, 6.625, 6.625]])

In [13]:
@numba.njit
def demo_iter(a_grid, phi, a_max, plc, pie):
    err = 1.
    n = 0
    
    demon = np.ones((601,7), dtype=np.float64) / (601*7)
    
    itv = (a_max+phi)/1200
    a_itv = np.empty((601,2), dtype=np.float64)
    a_itv[:,0] = a_grid - itv
    a_itv[:,1] = a_grid + itv
    
    while err > 1e-6 and n <= 1000:
        demo = demon.copy()
        temp = np.zeros((601*7,8), dtype=np.float64)
        demon = np.zeros((601,7), dtype=np.float64)
        for i in range(601):
            for j in range(7):
                temp[i*7+j,0]=plc[i,j]
                temp[i*7+j,1:8]=demo[i,j] * pie[j,:]
        
        for k in range(601):
            location = np.where(np.logical_and(np.greater_equal(temp[:,0],a_itv[k,0]), np.less(temp[:,0],a_itv[k,1])))
            for l in location:
                demon[k,:] = np.sum(temp[l,1:8], axis=0)
        
        if n % 10 == 0:        
            err = np.max(np.abs(demon - demo))
            
        n += 1
    return demon

In [14]:
err = 1.
count = 0
r = 0.01

#momentum = 0.15
#lr = 0.001
#decay = 0.98
r_low=-0.04
r_up=(1-beta)/beta
start = datetime.datetime.now()

while err >= 15e-4 and count <= 20:
    k = (((r+delta)/(alpha * L**(1-alpha)))) ** (1/(alpha-1))
    w = (1-alpha) * (k**alpha) * (L**(-alpha))
    
    if r > 0:
        phi = min(b, w*l_min/r)
    else:
        phi = b
    
    a_grid = np.linspace(-phi, a_max, 601, dtype=np.float64)
    plc = plc_iter(phi, a_max, beta, mu, piel, r, w, l_grid)
    demo = demo_iter(a_grid, phi, a_max, plc, piel)
    kprime = np.sum(demo*a_grid.reshape(-1,1))
    
    err = np.abs(k - kprime)
    if kprime < k:
        r_low = r
    else:
        r_up = r
    r = 0.5*(r_up+r_low)
    
    if count % 10 ==0:
        print(count)
    
    count += 1

end = datetime.datetime.now()
print(end-start)

0
10
20
0:05:15.249924


In [16]:
k, kprime, r, w

(6.390075882618307, 6.388724934722765, 0.03617273966471357, 1.2091523735138896)

In [15]:
k, kprime, r, w

(33.80696520502107,
 8.045887039317522e+301,
 -0.03999997615814209,
 2.2026276886679885)

In [13]:
w

1.0773325810396563

In [11]:
r

0.06263229728210717

array([[-3., -3., -3., ..., 11., 11., 11.],
       [-3., -3., -3., ..., 11., 11., 11.],
       [-3., -3., -3., ..., 11., 11., 11.],
       ...,
       [11., 11., 11., ..., 11., 11., 11.],
       [11., 11., 11., ..., 11., 11., 11.],
       [11., 25., 11., ..., 11., 11., 11.]])

In [12]:
r=0.01
w=1.2
a_grid = np.linspace(-3, a_max, 601, dtype=np.float64)
drn = np.repeat(a_grid.reshape(-1,1), 7, axis=1)
full = si.interp1d(a_grid,drn, axis=0, fill_value='extrapolate')

In [1]:
def obj_fct(ap, a, l, tras):
        res = ((1+r)*a+w*l-ap) ** (-mu) - beta * (1+r)* ((1+r)*ap + w*l_grid - full(ap)) ** (-mu) @tras
        return res